Pamiętaj o zmianie środowiska wykonawczego (GPU)

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [ ]:
dataset, info = tfds.load(
    "oxford_iiit_pet",  # https://www.tensorflow.org/datasets/catalog/oxford_iiit_pet
    with_info=True,
)

train_ds = dataset["train"]
test_ds = dataset["test"]

In [ ]:
# Podgląd segmentation masks
for sample in dataset["train"].take(5):
    mask = sample["segmentation_mask"]
    plt.imshow(tf.squeeze(mask))
    plt.colorbar()
    plt.title("Raw Mask Values")
    plt.show()

In [ ]:
IMG_SIZE = 128

def normalize(sample):
    image = sample["image"]
    mask = sample["segmentation_mask"]

    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0

    mask = tf.image.resize(
        mask,
        (IMG_SIZE, IMG_SIZE),
        method=tf.image.ResizeMethod.NEAREST_NEIGHBOR # resize masek tak, by zachowały się w nich wyłącznie wartości 1, 2, 3 (odpowiadające klasom pikseli: tło, granica, zwierzę)
    )
    mask = tf.cast(mask, tf.int32) - 1  # w masce numerki są od 1, my chcemy nasze klasy indeksować od 0 (dla zgodności z frameworkiem)

    return image, mask

train_ds = train_ds.map(normalize).batch(16).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.map(normalize).batch(16)

In [ ]:
for img, mask in train_ds.take(1):
    print(img.shape, mask.shape)

In [ ]:
# Budujemy model encoder - decoder
# Jako encoder bierzemy CNN (MobileNet), pomijając warstwę spłaszczającą
# Następnie jako decoder dodajemy parę warstw typu UpSampling, łącząc je z odpowiadającymi warstwami encodera (skip layers - podobnie jak w U-necie)
def unet_like_model():
    inputs = tf.keras.layers.Input((IMG_SIZE, IMG_SIZE, 3))

    base_model = tf.keras.applications.MobileNetV2(
        input_tensor=inputs,
        include_top=False, # pomijamy flatten
        weights="imagenet" # inicjalizujemy wagami ImageNet
    )

    skip_layers = [
        base_model.get_layer(name).output
        for name in [
            "block_1_expand_relu",
            "block_3_expand_relu",
            "block_6_expand_relu",
            "block_13_expand_relu",
            "block_16_project",
        ]
    ]

    x = skip_layers[-1]
    skips = reversed(skip_layers[:-1])

    for skip in skips:
        x = tf.keras.layers.UpSampling2D()(x)
        x = tf.keras.layers.Concatenate()([x, skip])

    x = tf.keras.layers.UpSampling2D(size=(2, 2))(x)
    outputs = tf.keras.layers.Conv2D(3, 1, activation="softmax")(x)

    return tf.keras.Model(inputs, outputs)

model = unet_like_model()
model.summary()

In [ ]:
tf.keras.utils.plot_model(  # opcjonalnie możemy wyrysować schemat modelu (tutaj jednak będzie to dość nieczytelne)
    model,
    show_shapes=True,
    expand_nested=True,
    dpi=120
)

In [ ]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",  # funkcja straty dla klas podanych jako int: https://keras.io/api/losses/probabilistic_losses/#sparsecategoricalcrossentropy-class
    metrics=["accuracy"]
)

model.fit(train_ds, epochs=5, validation_data=test_ds)

In [ ]:
def show_predictions(dataset):
    for images, masks in dataset.take(10):
        preds = model.predict(images)
        plt.figure(figsize=(12,4))

        plt.subplot(1,3,1)
        plt.imshow(images[0])
        plt.title("Image")

        plt.subplot(1,3,2)
        plt.imshow(tf.squeeze(masks[0]))
        plt.title("Mask")

        plt.subplot(1,3,3)
        plt.imshow(tf.argmax(preds[0], axis=-1))
        plt.title("Prediction")

        plt.show()

show_predictions(test_ds)